In [ ]:
sec = 10000
h, m_temp = divmod(sec, 3600)
m, s = divmod(m_temp, 60)
del m_temp
h, m, s

In [2]:
from time import sleep
import json
import funkcje as f
import subprocess
import re
from lxml import etree as et

In [3]:
DATA = dict()
sesja = f.prepare_session(DATA)    
f.main_loop(sesja["activities"])
DATA

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1m[1] During: typing of argument at /home/roodygibon/Documents/programowanie wszystko/python/skrypty/pierdoly/autotimer/AutoTimer/funkcje.py (85)[0m
[1m
File "funkcje.py", line 85:[0m
[1mdef prepare_session(data):
[1m    today = datetime.datetime.today().strftime("%Y-%m-%d %H:%M:%S")
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'dict'>[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/latest/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/latest/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://n

In [5]:
# sleep(2)
sesja = f.prepare_session(DATA)    
main_loop(sesja["activities"])
DATA

{'2020-09-18 16:21:27': {'tag': 'session',
  'start': '2020-09-18 16:21:27',
  'end': None,
  'activities': {'Desktop': {'tag': 'activity',
    'name': 'Desktop',
    'entries': {0: {'tag': 'entry',
      'start_time': '2020-09-18 16:21:27',
      'end_time': '2020-09-18 16:21:27'}}},
   'JupyterLab - Mozilla Firefox': {'tag': 'activity',
    'name': 'JupyterLab - Mozilla Firefox',
    'entries': {0: {'tag': 'entry',
      'start_time': '2020-09-18 16:21:27',
      'end_time': '2020-09-18 16:21:37'}}}}},
 '2020-09-18 16:22:25': {'tag': 'session',
  'start': '2020-09-18 16:22:25',
  'end': None,
  'activities': {'Desktop': {'tag': 'activity',
    'name': 'Desktop',
    'entries': {0: {'tag': 'entry',
      'start_time': '2020-09-18 16:22:25',
      'end_time': '2020-09-18 16:22:25'}}},
   'JupyterLab - Mozilla Firefox': {'tag': 'activity',
    'name': 'JupyterLab - Mozilla Firefox',
    'entries': {0: {'tag': 'entry',
      'start_time': '2020-09-18 16:22:25',
      'end_time': '2020-09

In [3]:
def printxml(x):
    print(et.tostring(x, pretty_print=True).decode("UTF-8"))

In [2]:
def make_activity(ID):
    NAME = get_active_window_title2(ID)
    return et.Element("activity", id=ID, name=NAME)

In [3]:
def make_entry(START, END):
    s = START.strftime("%Y-%m-%d %H:%M:%S")
    e = END.strftime("%Y-%m-%d %H:%M:%S")
    return et.Element("entry", start_time= s, end_time= e)

In [ ]:
def make_session(num, time):
    return et.Element("session",
                      id=num,
                      start=time,
                      end="None")

el = make_session("0", "222")
act = make_activity(b"0x38900", "name_act")
act2 = make_activity(b"0x38901", "name_act2")
times = [f.mytimer() for _ in range(4)]
en1 = make_entry(times[0], times[1])
en2 = make_entry(times[2], times[3])
en3 = make_entry(times[0], times[1])
en4 = make_entry(times[2], times[3])

el.append(act)
el.append(act2)

act.append(en1)
act.append(en2)

act2.append(en3)
act2.append(en4)

printxml(el)

In [9]:
for ch in el.getchildren():
    print(ch.get("id"))

0x38900
0x38901


In [90]:

            
# for i in range(3):
#     print(next(monitor_status))
#     sleep(1)
c =0
while monitor_is_on():
    print("hej")
    sleep(1)
    c+=1 
    if c > 3:
        break


hej
hej
hej
hej


In [50]:
ID = "0x38900"
act = el.xpath(f"activity[@id='{ID}']")#, {"id":"0x38900"})
act#.getchildren()
# printxml(act)

[<Element activity at 0x7fb6bf6d9e80>]

In [ ]:
def update_activity(activs, ID, start, end):
    activity = activs.xpath(f"activity[@id='{ID}']")
    if activity:
        activity = activity[0]
    else:
        activity = make_activity(ID)
        activs.append(activity) 
        
    entry = make_entry(start, end)
    activity.append(entry)
    

In [ ]:
def main_loop(activities):
    previous_window_id =  b"0x1200007"# 'Desktop'
    start_time = f.mytimer()
    for _ in range(10): # 60 * 10
        active_window_id = f.get_active_window_id()
        if previous_window_id != active_window_id:
            end_time = f.mytimer()
            f.update_activity(activities, previous_window_id, start_time, end_time)
            start_time = f.mytimer()
            
            previous_window_id = active_window_id
        sleep(1)
    
    end_time = f.mytimer()
    f.update_activity(activities, previous_window_id, start_time, end_time)
#     with open("outputs/filename.json", "w") as file:
#         json.dump(DATA, file, indent=4)
        

In [108]:
def get_ids_and_names_of_open_windows():
    with subprocess.Popen(['xprop', "-root", "_NET_CLIENT_LIST"], stdout=subprocess.PIPE) as window:
        stdout, stderr = window.communicate()
        
    matches = re.findall(b"(0x\w+)", stdout)
    return {ID: get_active_window_title2(ID) for ID in matches}
        
get_ids_and_names_of_open_windows()

{b'0xe00003': 'xfce4-panel',
 b'0x1200007': 'Desktop',
 b'0x4000003': '(309) Nuclear Winter (Multiple Dark Ambient Hours) - YouTube - Firefox Developer Edition',
 b'0x5c00003': 'Terminal - roodygibon@MAMALYGA:~',
 b'0x5e00003': 'JupyterLab - Mozilla Firefox',
 b'0x5c0018c': 'Terminal - roodygibon@MAMALYGA:~'}

In [106]:
def get_active_window_title2(ID):
    with subprocess.Popen(['xprop', '-id', ID, 'WM_NAME'], stdout=subprocess.PIPE) as window:
        stdout, stderr = window.communicate()
    match = re.match(b"WM_NAME\(\w+\) = (?P<name>.+)$", stdout)
    if match is not None:
        name = match.group("name").strip(b'"').decode("utf-8")
    return name or None

In [110]:
def get_active_window_id():
    with subprocess.Popen(['xprop', '-root', '_NET_ACTIVE_WINDOW'], stdout=subprocess.PIPE) as root:
        stdout, stderr = root.communicate()
    m = re.search(b'^_NET_ACTIVE_WINDOW.* ([\w]+)\, 0x\w+\n$', stdout)
    if m is None:
        return None
    window_id = m.group(1)
    return window_id

In [27]:
def func():
    with subprocess.Popen(['xprop', "-root", "_NET_ACTIVE_WINDOW"], stdout=subprocess.PIPE) as window:
        stdout, _ = window.communicate()
        return stdout
%timeit func()


4.47 ms ± 262 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
